In [1]:
#| default_exp ml_classification

# Water pixels classification

Steps :
1. Collect water pixels into table (consider negative buffer for mroe accuracy)  
2. consider - filter pixels based on the values I found in task 2  
3. create dataframe  
4. split x,y,train,test and train model  
5. evaluate the model

In [2]:
#|hide
from nbdev.showdoc import *

In [10]:
#|export
#|hide

import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# import geopandas as gpd
# import matplotlib.pyplot as plt


from flood_exercise import utils_func
from flood_exercise import const_vals as CONST

In [7]:
#|export

class classification_pixels():
  
  def __init__(self,
               path_labels_str : str , #path to the labeled images
               path_imgs_str : str , #path to S2 images to be used for train
               target_col : str , #name of the target attribute in the dataframe. Will be used for model training
               cols_to_drop : list = [None] , # list of columns to be dropped from the dataframe. These columns won't be used to train the model. 
               test_size : float=0.25 , #size of test data
               ):
    
    self.target_col = target_col
    self.cols_to_drop = cols_to_drop
    self.test_size = test_size
    
    self.path_labels = utils_func.load_list_paths(path = path_labels_str , filter_file = True)
    self.path_imgs = utils_func.load_list_paths(path = path_imgs_str , filter_file= True)

    dfs_pixels = []
    for label_path in self.path_labels:
      id_path = label_path.split(CONST.SPLIT_TILES_NAMES_STR1)[-1].split(CONST.SPLIT_TILES_NAMES_STR2)[1]
      
      #find the matching S2 image
      s2_img_path = [x for x in self.path_imgs if id_path in x]
      
      if len(s2_img_path)==1:

        s2_img = rasterio.open(s2_img_path[0]).read()
        cols = rasterio.open(s2_img_path[0]).descriptions + rasterio.open(label_path).descriptions
        cols = [str(x) for x in cols]
        label_img = rasterio.open(label_path).read()
        

        # Stack the imags 
        stacked_img = np.concatenate((s2_img, label_img), axis=0)
        
        df_pixels = pd.DataFrame(stacked_img.reshape([stacked_img.shape[0],-1]).T)
        df_pixels.columns = cols
        dfs_pixels.append(df_pixels)

      else:
        continue

    self.df_res = pd.concat(dfs_pixels)


  
  def _prepare_dataframe_for_train_(self):

    if self.cols_to_drop[0] != None :
      self.df_res.drop(self.cols_to_drop,axis=1,inplace=True)

    x = self.df_res.drop(self.target_col,axis=1,inplace=True)
    y = self.df_res[self.target_col].values

    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        test_size = self.test_size, 
                                                        random_state = CONST.RANDOM_STATE)


    


    

In [8]:
instance = classification_pixels(path_imgs_str=r"D:\git\flood_exercise\S2",
                                 path_labels_str=r"D:\git\flood_exercise\S2_HANDLABELED",
                                 target_col = 'None',
                                 cols_to_drop= ['qc']
                                 )
instance.df_res

,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12,None,qc
0,1518,1233,1186,875,1256,2642,3335,3061,3561,342,35,2394,1158,0.0,NaN
1,1518,1239,1194,897,1256,2642,3335,2887,3561,342,35,2394,1158,0.0,NaN
2,1517,1251,1253,933,1381,2806,3312,3115,3806,360,33,2563,1387,0.0,NaN
3,1517,1255,1247,936,1381,2806,3312,3181,3806,360,33,2563,1387,0.0,NaN
4,1517,1228,1197,897,1288,2498,3048,2898,3362,360,33,2493,1375,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262139,1178,1180,1213,1120,1399,2637,2967,2829,3350,1006,12,2224,1241,0.0,NaN
262140,1178,1121,1148,976,1427,2444,3041,2860,3306,1006,12,2264,1325,0.0,NaN
262141,1178,1097,1148,1000,1427,2444,3041,2705,3306,1006,12,2264,1325,0.0,NaN
262142,1178,1097,1148,1000,1427,2444,3041,2705,3306,1006,12,2264,1325,0.0,NaN


In [ ]:
instance.df_res.columns

In [ ]:
pd.DataFrame(test.reshape([test.shape[0],-1]).T)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(test[13,:,:])